In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
# import lightgbm as lgb
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB


import pickle
import sys,os
from all_model_py import CutDebt, IDClassifier, IfKnowDebtor, Installment, WillingToPay, ConfirmLoan

# import sys
# sys.path.append('../CutDebt/')
# sys.path.append('../IDClassifier/')
# sys.path.append('../IfKnowDebtor/')
# sys.path.append('../Installment/')
# sys.path.append('../SetDueDay/')
# sys.path.append('../WillingToPay/')

# from CutDebt_py import CutDebt
# from IDClassifier_py import IDClassifier
# from IfKnowDebtor_py import IfKnowDebtor
# from Installment_py import Installment
# from SetDueDay_py import SetDueDay
# from WillingToPay_py import WillingToPay

In [2]:
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
func_list = [CutDebt,IDClassifier,IfKnowDebtor,Installment,ConfirmLoan,WillingToPay]

ind = 0
for each_model in model_list:
    print(each_model)
    
    # load data
    path = '../../data/{}/'
    data_train = pd.read_csv(path.format(each_model) + 'cleaned_mock_up_data.csv', encoding='utf8')
    print(data_train.label.value_counts())
    
    
    ######## remove time pattern from split_text
    
    # get tfidf
    phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

    print('fitting phrase')
    phrase_vectorizer.fit(data_train.split_text)

    print('transform phrase')
    phrase = phrase_vectorizer.transform(data_train.split_text)

    
    # linear svc
    l_svc = LinearSVC()
    lsvc = CalibratedClassifierCV(l_svc) 
    lsvc.fit(phrase, data_train.label)
    
    
    # logistic
    log_r = LogisticRegression()
    log_r.fit(phrase, data_train.label)
    
    
    # Naive Bayes
    naive_b = MultinomialNB()
    naive_b.fit(phrase, data_train.label)
    
    
    print('finish training')
    
    
    
    
    
#     for threhold
    
    # save model
    save_path = '../../savedModel/{}/'
#     # save tfidf
#     pickle.dump(phrase_vectorizer, open(save_path.format(each_model) + "tfidf.pickle", "wb"))
#     # save linear svc
#     pickle.dump(lsvc, open(save_path.format(each_model) + "LinearSVC.pickle", "wb"))
#     # save logistic
#     pickle.dump(log_r, open(save_path.format(each_model) + "Logistic.pickle", "wb"))
#     # save lightGBM
# #     pickle.dump(lgbm_model, open(save_path.format(each_model) + "Lgbm.pickle", "wb"))
#     # save naive bayes
#     pickle.dump(naive_b, open(save_path.format(each_model) + "nb.pickle", "wb"))

    
    other = pickle.load(open('../../savedModel/others/{}/{}_other.pickle'.format(each_model,each_model),'rb'))
    
    func = func_list[ind]
    result = func(svc=lsvc, logistic=log_r, nb=naive_b, tfidf=phrase_vectorizer, other=other,  jieba_path='../WordCut/userdict.txt')
    pickle.dump(result, open(save_path.format(each_model) + each_model + '.pickle', "wb"))
    ind = ind + 1
    
    
    
    

CutDebt
0    4105
2    2197
1    2144
Name: label, dtype: int64
fitting phrase
transform phrase


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.499 seconds.


finish training


Prefix dict has been built succesfully.


IDClassifier
0    1446
2    1322
1    1296
Name: label, dtype: int64
fitting phrase
transform phrase
finish training
IfKnowDebtor
0    1483
2    1317
1    1177
Name: label, dtype: int64
fitting phrase
transform phrase
finish training
Installment
0    4105
2    2222
1    2034
Name: label, dtype: int64
fitting phrase
transform phrase
finish training
ConfirmLoan
0    580
1    553
2    438
Name: label, dtype: int64
fitting phrase
transform phrase
finish training
WillingToPay
0    2106
1    2095
2    2069
3    1294
Name: label, dtype: int64
fitting phrase
transform phrase
finish training


In [3]:
# import jieba
# sentence = '好'
# sentence = jieba.cut(sentence)
# sentence = ' '.join(sentence)
# sentence = phrase_vectorizer.transform([sentence])
# naive_b.predict_proba(sentence)

In [4]:
idc = pickle.load(open("../../savedModel/IDClassifier/IDClassifier.pickle", 'rb'))
cutd = pickle.load(open("../../savedModel/CutDebt/CutDebt.pickle", 'rb'))
ifk = pickle.load(open("../../savedModel/IfKnowDebtor/IfKnowDebtor.pickle", 'rb'))
will = pickle.load(open("../../savedModel/WillingToPay/WillingToPay.pickle", 'rb'))
inst = pickle.load(open("../../savedModel/Installment/Installment.pickle", 'rb'))
conf = pickle.load(open("../../savedModel/ConfirmLoan/ConfirmLoan.pickle", 'rb'))

In [5]:
will.re_time.process('下个月初还')

[{'pattern': '下个月初',
  'time': datetime.datetime(2018, 8, 5, 19, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
  'gapS': 2684204.597209,
  'gapH': 745.6123881136111}]

In [6]:
cutd.classify('3个月以后还',lower_bounder=36,upper_bounder=24*3)

{'label': 1, 'pred_prob': array([[0.28783117, 0.67684388, 0.03532495],
        [0.48451991, 0.43331745, 0.08216263],
        [0.42711315, 0.45029952, 0.12258733]]), 'av_pred': array([0.39982141, 0.52015362, 0.08002497]), 'add_sentiment': 1, 'time_extract': [{'pattern': '3个月',
   'time': datetime.datetime(2018, 10, 5, 19, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
   'gapS': 7954603.260945,
   'gapH': 2209.6120169291667}]}

In [7]:
will.classify('我明天不能还后天可以',lower_bounder=36,upper_bounder=24*3)

{'label': 10,
 'pred_prob': 1.0,
 'av_pred': 1.0,
 'time_extract': [{'pattern': '明天',
   'time': datetime.datetime(2018, 7, 6, 16, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
   'gapS': 81402.485664,
   'gapH': 22.611801573333334},
  {'pattern': '后天',
   'time': datetime.datetime(2018, 7, 7, 16, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
   'gapS': 167802.485664,
   'gapH': 46.61180157333334}]}

In [8]:
will.re_time.remove_time('我明天还不了')

'我 还不了'

In [25]:
conf.classify('我只借过5000')

{'label': 1, 'pred_prob': array([[0.03012535, 0.83516209, 0.13471256],
        [0.19332805, 0.47517069, 0.33150125],
        [0.18685578, 0.5741989 , 0.23894532]]), 'av_pred': array([0.13676973, 0.62817723, 0.23505304]), 'time_extract': []}